<a href="https://colab.research.google.com/github/AlbertBannister/cricinfo-commentary-scraper/blob/main/train_cricket_mlm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers accelerate datasets nvidia-ml-py3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 45.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 94.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.2 MB/s eta 0:00:00
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19171 sha256=1456bb269cd08f4c067aade28d4fc410daaa49ee9d3aae953143cc8734a477c9
  Stored in directory: /root/.cache/pip/wheels/5c/d8/c0/46899f8be7a75a

In [ ]:
import transformers
from datasets import load_dataset, ReadInstruction
from transformers import AutoTokenizer
model_name = "bert-base-uncased"

In [ ]:
from pynvml import *


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

In [ ]:
print_gpu_utilization()

GPU memory occupied: 601 MB.


In [ ]:
import torch
torch.ones((1,1)).to("cuda")

tensor([[1.]], device='cuda:0')

In [ ]:
data=load_dataset("text", data_files="/content/drive/MyDrive/Colab Notebooks/data/NLP/sense2vec_train.txt", split=ReadInstruction("train", to=100000))
# data=load_dataset("text", data_files="/content/drive/MyDrive/Colab Notebooks/data/NLP/sense2vec_train.txt")
dataset=data.train_test_split()
dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 75000
    })
    test: Dataset({
        features: ['text'],
        num_rows: 25000
    })
})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["text"]])

In [ ]:
tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=dataset["train"].column_names,
)

Map (num_proc=4):   0%|          | 0/75000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (521 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (661 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (616 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (521 > 512). Running this sequence through the model will result in indexing errors


Map (num_proc=4):   0%|          | 0/25000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (518 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (520 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (617 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
block_size = 512


def group_texts(examples):

    # Concatenate all texts.

    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}

    total_length = len(concatenated_examples[list(examples.keys())[0]])

    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can

    # customize this part to your needs.

    if total_length >= block_size:

        total_length = (total_length // block_size) * block_size

    # Split by chunks of block_size.

    result = {

        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]

        for k, t in concatenated_examples.items()

    }

    return result

In [ ]:
lm_dataset = tokenized_dataset.map(group_texts, batched=True, num_proc=4)
lm_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 10857
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3587
    })
})

In [ ]:
from transformers import DataCollatorForLanguageModeling

# tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [ ]:
from transformers import AutoModelForMaskedLM, TrainingArguments, Trainer

model = AutoModelForMaskedLM.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/my_models/cricket_mlm",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=10,
    save_strategy="epoch",
    weight_decay=0.01,
    per_device_train_batch_size=16,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,1.436400,0.839380
2,0.917800,0.653361
3,0.688400,0.570495
4,0.633600,0.518272
5,0.588900,0.475513
6,0.535900,0.455644
7,0.516000,0.434868
8,0.508400,0.424699
9,0.488100,0.413286
10,0.480800,0.415889


TrainOutput(global_step=6790, training_loss=0.655784833905272, metrics={'train_runtime': 4362.5092, 'train_samples_per_second': 24.887, 'train_steps_per_second': 1.556, 'total_flos': 2.8576146525696e+16, 'train_loss': 0.655784833905272, 'epoch': 10.0})

In [ ]:
trainer.save_model("/content/drive/MyDrive/my_models/cricket_mlm")